In [1]:
from src import Tensor,cross_entropy,Adam,SGD,Linear, Module, Sequential, ReLU,dataloader
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [3]:
class Net(Module):
    def __init__(self, outc):
        super().__init__()
        self.seq = Sequential(
            Linear(None, 64, bias=True),
            ReLU(),
            Linear(None, 64*2, bias=True),
            ReLU(),
            Linear(None, outc),
        )

    def forward(self, x):
        x = self.seq(x)
        return x


net = Net(len(np.unique(train_y)))
optimizer = Adam(net.trainable_params(), lr=0.001,amsgrad=True)
for epoch in range(10):  # loop over the dataset multiple times
    losses = []
    accs = []
    for X, y in dataloader(64*2, train_X, train_y):

        # forward + backward + optimize
        outputs = net.forward(X)
        loss = cross_entropy(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        assert loss.item() > 0, loss
        optimizer.step()

        losses.append(loss.item())
        accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
    print(
        f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):.3f} accuracy: {np.mean(accs)*100:3.2f}')

print('Finished Training')

[Epoch   1] loss: 2.795 accuracy: 61.04
[Epoch   2] loss: 0.388 accuracy: 88.67
[Epoch   3] loss: 0.222 accuracy: 92.68
[Epoch   4] loss: 0.176 accuracy: 93.46
[Epoch   5] loss: 0.121 accuracy: 96.25
[Epoch   6] loss: 0.112 accuracy: 96.74
[Epoch   7] loss: 0.096 accuracy: 97.34
[Epoch   8] loss: 0.040 accuracy: 98.59
[Epoch   9] loss: 0.034 accuracy: 99.22
[Epoch  10] loss: 0.036 accuracy: 98.96
Finished Training


In [4]:
(train_y == net(Tensor(train_X)).data.argmax(-1)).mean()

0.9922092376182526